## Trabalho Final

Carregue os arquivos no Hadoop/Hive, em seguida construa as seguintes analises:

1) Total de acidentes com vítima por bairro em acidentes com embriaguez;

2) Total de acidentes por tipo de pavimento e condição do tempo;

3) Total de pessoas acidentadas por tipo de veiculo e tipo de pavimentação;

4) Média de idade dos condutores por tipo de veículo e tipo de acidente;

5) Média de idade dos condutores por indicativo de embriaguez;

In [107]:
# carregando dados

df_env = spark.read.csv("hdfs://node1:8020//user/vagrant/dados/trabalho-final/si_env-2019.csv", sep=';', header=True, encoding = 'latin1')
df_bol = spark.read.csv("hdfs://node1:8020//user/vagrant/dados/trabalho-final/si-bol-2019.csv", sep=';', header=True, encoding = 'latin1')
df_log = spark.read.csv("hdfs://node1:8020//user/vagrant/dados/trabalho-final/si-log-2019.csv", sep=';', header=True, encoding = 'latin1')

In [108]:
# fazendo join das tabelas

df_join = df_env.join(df_bol, df_env['num_boletim'] == df_bol[' NUMERO_BOLETIM'], 'left').join(df_log, df_env['num_boletim'] == df_log['Nº_boletim'], 'left')

In [109]:
# importando libs

import pyspark.sql.functions as F

In [110]:
# 1 Total de acidentes com vítima por bairro em acidentes com embriagues

df_resposta1 = df_join.filter(df_join[' DESC_TIPO_ACIDENTE'].like('%COM VITIMA%') | df_join[' DESC_TIPO_ACIDENTE'].like('%C/ VITIMA%')).filter(df_join[' Embreagues'] == 'SIM')
df_resposta1 = df_resposta1.groupby('nome_bairro').agg(F.count('*').alias('total_acidentes'))
df_resposta1.orderBy(F.col('total_acidentes').desc()).show(10, 0)

+--------------------------------------------------+---------------+
|nome_bairro                                       |total_acidentes|
+--------------------------------------------------+---------------+
|SANTA EFIGENIA                                    |17             |
|CENTRO                                            |13             |
|LOURDES                                           |11             |
|CIDADE NOVA                                       |9              |
|PLANALTO                                          |9              |
|HELIOPOLIS                                        |8              |
|CARLOS PRATES                                     |8              |
|SAVASSI                                           |8              |
|TIROL                                             |7              |
|COPACABANA                                        |7              |
+--------------------------------------------------+---------------+
only showing top 10 rows



In [111]:
# 2 Total de acidentes por tipo de pavimento e condição do tempo

df_resposta2 = df_join.groupby(' PAVIMENTO', ' DESC_TEMPO').agg(F.count('*').alias('total_acidentes'))
df_resposta2.orderBy(F.col('total_acidentes').desc()).show(10, 0)

+---------------+---------------+---------------+
| PAVIMENTO     | DESC_TEMPO    |total_acidentes|
+---------------+---------------+---------------+
|NAO INFORMADO  |NAO INFORMADO  |18884          |
|ASFALTO        |BOM            |16535          |
|ASFALTO        |CHUVA          |1358           |
|ASFALTO        |NUBLADO        |825            |
|ASFALTO        |NAO INFORMADO  |336            |
|CONCRETO       |BOM            |129            |
|NAO INFORMADO  |BOM            |124            |
|CALCAMENTO     |BOM            |109            |
|CONCRETO       |NUBLADO        |52             |
|ASFALTO        |NEBLINA        |19             |
+---------------+---------------+---------------+
only showing top 10 rows



In [112]:
# 3 Total de pessoas acidentadas por tipo de veiculo e tipo de pavimentação

df_resposta3 = df_join.groupby('especie_veiculo', ' PAVIMENTO').agg(F.sum(' Nº_envolvido').alias('total_acidentados'))
df_resposta3.orderBy(F.col('total_acidentados').desc()).show(10, 0)

+------------------------------+---------------+-----------------+
|especie_veiculo               | PAVIMENTO     |total_acidentados|
+------------------------------+---------------+-----------------+
|AUTOMOVEL                     |ASFALTO        |16835.0          |
|AUTOMOVEL                     |NAO INFORMADO  |13659.0          |
|MOTOCICLETA                   |NAO INFORMADO  |11936.0          |
|MOTOCICLETA                   |ASFALTO        |11479.0          |
|                              |NAO INFORMADO  |2204.0           |
|ONIBUS                        |ASFALTO        |2085.0           |
|ONIBUS                        |NAO INFORMADO  |2001.0           |
|                              |ASFALTO        |1697.0           |
|CAMINHONETE                   |ASFALTO        |957.0            |
|ONIBUS                        |CONCRETO       |827.0            |
+------------------------------+---------------+-----------------+
only showing top 10 rows



In [115]:
# 4 Média de idade dos condutores por tipo de veículo e tipo de acidente

df_resposta4 = df_join.filter(df_join[' condutor'] == 'S').groupby('especie_veiculo', ' DESC_TIPO_ACIDENTE').agg(F.avg(' Idade').alias('media_idade'))
df_resposta4 = df_resposta4.withColumn("media_idade", F.round(df_resposta4["media_idade"], 2))
df_resposta4.orderBy(F.col('especie_veiculo').asc()).show(10, 0)

+------------------------------+--------------------------------------------------+-----------+
|especie_veiculo               | DESC_TIPO_ACIDENTE                               |media_idade|
+------------------------------+--------------------------------------------------+-----------+
|                              |QUEDA DE PESSOA DE VEICULO                        |34.0       |
|AUTOMOVEL                     |QUEDA DE VEICULO COM VITIMA                       |38.3       |
|AUTOMOVEL                     |ATROPELAMENTO DE PESSOA SEM VITIMA FATAL          |35.84      |
|AUTOMOVEL                     |ATROPELAMENTO DE PESSOA COM VITIMA FATAL          |30.81      |
|AUTOMOVEL                     |ABALROAMENTO COM VITIMA                           |37.87      |
|AUTOMOVEL                     |ATROPELAMENTO DE ANIMAL COM VITIMA                |25.0       |
|AUTOMOVEL                     |COLISAO DE VEICULOS COM VITIMA                    |37.69      |
|AUTOMOVEL                     |CAPOTAME

In [114]:
# 5 Média de idade dos condutores por indicativo de embriaguez

df_resposta5 = df_join.filter(df_join[' condutor'] == 'S').groupby(' Embreagues').agg(F.avg(' Idade').alias('media_idade_condutores'))
df_resposta5 = df_resposta5.withColumn("media_idade_condutores", F.round(df_resposta5["media_idade_condutores"], 2))
df_resposta5.show(truncate=0)

+-------------+----------------------+
| Embreagues  |media_idade_condutores|
+-------------+----------------------+
|NÃO          |36.91                 |
|SIM          |36.52                 |
|NÃO INFORMADO|0.95                  |
+-------------+----------------------+

